# Q1

In [20]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

In [13]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Get weather data for a specific city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city to query the weather data"
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

## Test it

In [14]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# Cargar .env
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [15]:
import chat_assistant

tools = chat_assistant.Tools()
tools.add_tool(get_weather, get_weather_tool)

In [16]:
tools.get_tools()

[{'type': 'function',
  'name': 'get_weather',
  'description': 'Get weather data for a specific city',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'The city to query the weather data'}},
   'required': ['city'],
   'additionalProperties': False}}]

In [17]:
openai_client = OpenAI(api_key=api_key)

In [18]:

developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=openai_client
)

In [19]:
chat.run()

Chat ended.


## Add another tool

In [28]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [36]:
set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "Record the weather for a specific city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city to set the weather data"
            },
            "temp": {
                "type": "number",
                "description": "The temperature for the city"
            },
        },
        "required": ["city","temp"],
        "additionalProperties": False
    }
}

In [37]:
known_weather_data

{'berlin': 20.0}

In [40]:
tools.get_tools()

[{'type': 'function',
  'name': 'get_weather',
  'description': 'Get weather data for a specific city',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'The city to query the weather data'}},
   'required': ['city'],
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'set_weather',
  'description': 'Record the weather for a specific city',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'The city to set the weather data'},
    'temp': {'type': 'number', 'description': 'The temperature for the city'}},
   'required': ['city', 'temp'],
   'additionalProperties': False}}]

In [39]:
tools.add_tool(set_weather, set_weather_tool)

In [41]:
chat.run()

Chat ended.


In [42]:
known_weather_data

{'berlin': 25}

# Q6. MCP Client interaction

In [8]:
from fastmcp import Client
import weather_server

async def main():
    async with Client(weather_server.mcp) as mcp_client:
        tools = await mcp_client.list_tools()
        print(tools)


In [ ]:
await main()

[Tool(name='get_weather', title=None, description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta=None), Tool(name='set_weather', title=None, description="Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.", inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'temp': {'title': 'Te